In [ ]:
from zipfile import ZipFile
file_name="/content/99505_234911_bundle_archive.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
batch_size=32
img_height=200
img_width=200

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
datagen_train = ImageDataGenerator()
datagen_validation = ImageDataGenerator()

#td= tf.keras.preprocessing.image_dataset_from_directory("images/", validation_split=0.2, subset="training", seed=123, image_size=(img_height,img_width),batch_size=batch_size) 

In [ ]:
#vd= tf.keras.preprocessing.image_dataset_from_directory("images/", validation_split=0.2, subset="validation", seed=123, image_size=(img_height,img_width),batch_size=batch_size) 

In [ ]:
train_generator = datagen_train.flow_from_directory("images/train",
                                                    target_size=(img_height,img_width),
                                                    
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

validation_generator = datagen_validation.flow_from_directory("images/validation",
                                                    target_size=(img_height,img_width),
                                                    
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [ ]:
for image_batch, labels_batch in train_generator:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 200, 200, 3)
(32, 7)


In [ ]:
# normalization_images =tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
# normalised_td=train_generator.map(lambda x,y:(normalization_images(x),y))
# image_batch, labels_batch= next(iter(normalised_td))

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [ ]:
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width,3)))

In [ ]:
model.add(tf.keras.layers.Conv2D(32,3,padding="same",activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32,3,activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(64,3,padding="same",activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64,3,activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.35))
model.add(tf.keras.layers.Conv2D(128,3,padding="same",activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128,3,activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.35))
model.add(tf.keras.layers.Conv2D(256,3,activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.35))


model.add(tf.keras.layers.Flatten())

In [ ]:
# model.add(tf.keras.layers.LeakyReLU(alpha=0.5))
# model.add(tf.keras.layers.Dropout(0.1))
# model.add(tf.keras.layers.Dense(64,activation="relu",kernel_regularizer=tf.keras.regularizers.L1(0.01)))
# model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(7,activation="softmax"))


In [ ]:
# import tensorflow.keras.optimizers.Adam
import tensorflow

model.compile(optimizer="adam",loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_2 (Rescaling)      (None, 200, 200, 3)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 200, 200, 32)      896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 200, 200, 32)      128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 198, 198, 32)      9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 99, 99, 64)       

In [ ]:
# model.fit(train_generator,epochs=50)
history = model.fit_generator(generator=train_generator,
                                
                                epochs=50,
                                # validation_data = validation_generator,
                                
                                )


Epoch 1/50
901/901 [==============================] - 57s 63ms/step - loss: 1.8297 - accuracy: 0.2827
Epoch 2/50
901/901 [==============================] - 57s 63ms/step - loss: 1.6900 - accuracy: 0.3293
Epoch 3/50
901/901 [==============================] - 57s 63ms/step - loss: 1.6243 - accuracy: 0.3594
Epoch 4/50
901/901 [==============================] - 57s 63ms/step - loss: 1.5187 - accuracy: 0.4072
Epoch 5/50
901/901 [==============================] - 57s 63ms/step - loss: 1.4200 - accuracy: 0.4510
Epoch 6/50
901/901 [==============================] - 57s 63ms/step - loss: 1.3543 - accuracy: 0.4796
Epoch 7/50
901/901 [==============================] - 57s 63ms/step - loss: 1.2965 - accuracy: 0.5050
Epoch 8/50
901/901 [==============================] - 57s 63ms/step - loss: 1.2525 - accuracy: 0.5236
Epoch 9/50
901/901 [==============================] - 57s 63ms/step - loss: 1.2100 - accuracy: 0.5442
Epoch 10/50
901/901 [==============================] - 57s 63ms/step - loss: 1.173

In [ ]:
model.evaluate(validation_generator)

In [ ]:
model.save("emoji.h5")